In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 6)

In [ ]:
df = pd.read_csv("../input/train.csv")

In [ ]:
df.head().T

In [ ]:
df.shape

In [ ]:
print (df.Store.unique())
print (df.Store.nunique())

In [ ]:
df['Year'] = df.Date.map(lambda x: int(x[:4]))

In [ ]:
df['Month']  =df.Date.map(lambda x: int(x[5:7]))

In [ ]:
df.head().T

In [ ]:
df.Promo.unique()

In [ ]:
df.StateHoliday.unique()

In [ ]:
df.groupby(['StateHoliday'])['Open'].agg('mean')

Видно, что когда StateHoliday равен 0 или "0", то магазины скорее всего открыты. В остальных случаях закрыты. Сделаем значения этого столбца числовыми, и сгруппируем все выходные в 1, т.к. магазины все равно не работают. 

In [ ]:
df.StateHoliday = df['StateHoliday'].map(lambda x: (int)(x in ['a','b','c']))

In [ ]:
df.StateHoliday.unique()

In [ ]:
df.groupby('StateHoliday')['Open'].mean()

In [ ]:
df.SchoolHoliday.unique()

In [ ]:
df[['StateHoliday', 'SchoolHoliday']].corr()

In [ ]:
df.corr()

In [ ]:
df[['Sales', 'Customers']].corr()

Ух, корреляция прям зашкаливает. Видно столбцы то зависимы.

In [ ]:
plt.subplot('121')
df['Sales'].hist(bins=100)
plt.subplot('122')
df['Customers'].hist(bins=100)

In [ ]:
df.DayOfWeek.unique()

In [ ]:
df.DayOfWeek[df.Open == 1].value_counts()

In [ ]:
y = df.groupby('DayOfWeek')['Sales'].sum()

In [ ]:
y

In [ ]:
sns.barplot(x=np.arange(7)+1, y=y)

Ну в понедельник и пятницу продажи побольше, в вс не работают магазины => нет продаж

In [ ]:
df[['Promo', 'Sales']].corr()

In [ ]:
xx = df.groupby(['Promo', 'Month'])['Sales'].sum()
xx

In [ ]:
plt.plot(xx[0], 'g', label='no promo')
plt.plot(xx[1], 'b', label='promo')

Видно, что обычно доход от промо больше чем обычный

In [ ]:
yms = df.groupby(['Year', 'Month'])['Sales'].sum()

In [ ]:
sns.barplot(x=np.arange(12)+1, y=yms[2013].sort_index())

In [ ]:
sns.barplot(x=np.arange(12)+1, y=yms[2014].sort_index())

In [ ]:
sns.barplot(x=np.arange(12)+1, y=yms[2014].sort_index() + yms[2013].sort_index())

В декабре продаж обычно больше всего, также в начале весны, лета продажи обычно поднимаются

In [ ]:
df_store = pd.read_csv("../input/store.csv")

In [ ]:
df_store.head().T

In [ ]:
df_store.StoreType.unique()

In [ ]:
df_store.Assortment.unique()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [ ]:
df_store.fillna(value=0, inplace=True)

In [ ]:
le = LabelEncoder()

In [ ]:
df_store.StoreType = le.fit_transform(df_store.StoreType)

In [ ]:
df_store.Assortment = le.fit_transform(df_store.Assortment)

In [ ]:
df_store.PromoInterval.unique()

In [ ]:
dict1 = {0: 0, 'Jan,Apr,Jul,Oct': 1, 'Feb,May,Aug,Nov': 2, 'Mar,Jun,Sept,Dec': 3}

In [ ]:
df_store.PromoInterval = df_store.PromoInterval.apply(lambda x: dict1[x], convert_dtype=False)

In [ ]:
df_store.set_index(np.arange(len(df_store))+1, inplace=True)

In [ ]:
df_store.drop('Store', axis=1, inplace=True)

In [ ]:
df_store.head().T

Добавим инфо о типе магазина, ассортименте и конкурентах к основной таблице

In [ ]:
df = df.join(df_store, on='Store')

In [ ]:
df.head().T

In [ ]:
df.dtypes

In [ ]:
df.PromoInterval = df.PromoInterval.map(lambda x: int(x))

In [ ]:
print (df['StoreType'].value_counts() / len(df))

In [ ]:
print (df['Assortment'].value_counts() / len(df))

Посмотрим на влияние появления конкурента на продажи

In [ ]:
sales8 = df.groupby(['Store', 'Year', 'Month'])['Sales'].mean()[8]

In [ ]:
plt.plot(range(31), sales8.values)
plt.plot([df.loc[7].CompetitionOpenSinceMonth+12, df.loc[7].CompetitionOpenSinceMonth+12], [sales8.min(), sales8.max()])

Заметен спад продаж после открытия конкурентов

In [ ]:
df_t = df.drop(['Customers', 'Date', 'Year'], axis=1)

In [ ]:
df_t.head(10).T

Когда магазин закрыт, то продаж нет. Уберем из обучения все такие записи и тогда столбец 'Open' тоже можно будет убрать. 

In [ ]:
df_t = df_t[df_t.Open == 1]

In [ ]:
df_t.shape

In [ ]:
df_t.drop('Open', axis=1, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import validation_curve

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_t.drop('Sales', axis=1).values, df_t.Sales.values,\
                                                    test_size = 0.2)

Подберем параметры для модели

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
params = range(50, 201, 50)

In [ ]:
scores, tst_scr = validation_curve(RandomForestRegressor(), X_train[:20000],\
                                   y_train[:20000], 'n_estimators', params, \
                                   cv=5, scoring='r2', verbose=2)

In [ ]:
scores_mean = scores.mean(axis=1)
scores_std = scores.std(axis=1)
tst_scr_mean = tst_scr.mean(axis=1)
tst_scr_std = tst_scr.std(axis=1)
plt.plot(params, tst_scr_mean)
plt.fill_between(params, tst_scr_mean + tst_scr_std, tst_scr_mean - tst_scr_std, alpha=0.3)
plt.plot(params, scores_mean)
plt.fill_between(params, scores_mean + scores_std, scores_mean - scores_std, alpha=0.3)

In [ ]:
X_train.shape[1]

In [ ]:
params = range(3, 9)

In [ ]:
scores, tst_scr = validation_curve(RandomForestRegressor(n_estimators=100), X_train[:20000], \
                                   y_train[:20000], 'max_features', params, \
                                   cv=3, scoring='r2', verbose=2)

In [ ]:
scores_mean = scores.mean(axis=1)
scores_std = scores.std(axis=1)
tst_scr_mean = tst_scr.mean(axis=1)
tst_scr_std = tst_scr.std(axis=1)
plt.plot(params, tst_scr_mean)
plt.fill_between(params, tst_scr_mean + tst_scr_std, tst_scr_mean - tst_scr_std, alpha=0.3)
plt.plot(params, scores_mean)
plt.fill_between(params, scores_mean + scores_std, scores_mean - scores_std, alpha=0.3)

In [ ]:
params = range(5, 51, 5)

In [ ]:
scores, tst_scr = validation_curve(RandomForestRegressor(n_estimators=100), X_train[:20000], \
                                   y_train[:20000], 'max_depth', params, \
                                   cv=3, scoring='r2', verbose=2)

In [ ]:
scores_mean = scores.mean(axis=1)
scores_std = scores.std(axis=1)
tst_scr_mean = tst_scr.mean(axis=1)
tst_scr_std = tst_scr.std(axis=1)
plt.plot(params, tst_scr_mean)
plt.fill_between(params, tst_scr_mean + tst_scr_std, tst_scr_mean - tst_scr_std, alpha=0.3)
plt.plot(params, scores_mean)
plt.fill_between(params, scores_mean + scores_std, scores_mean - scores_std, alpha=0.3)

In [ ]:
model1 = RandomForestRegressor(n_estimators=100, max_depth=20, n_jobs=4, verbose=2)

In [ ]:
model1.fit(X_train, y_train)

In [ ]:
idx = model1.feature_importances_.argsort()[::-1]

In [ ]:
ax = sns.barplot(x=df_t.drop('Sales', axis=1).columns[idx], y=model1.feature_importances_[idx])
_ = plt.setp(ax.get_xticklabels(), rotation=-90)

In [ ]:
y_pred = model1.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [ ]:
mean_absolute_error(y_test, y_pred)

In [ ]:
r2_score(y_test, y_pred)

Загружаем тестовые данные и приводим их к нужному виду для модели. 

In [ ]:
df_test = pd.read_csv('../input/test.csv')

In [ ]:
df_test['Year'] = df_test.Date.map(lambda x: int(x[:4]))

In [ ]:
df_test['Month'] = df_test.Date.map(lambda x: int(x[5:7]))

In [ ]:
df_test.StateHoliday = df_test['StateHoliday'].map(lambda x: (int)(x in ['a','b','c']))

In [ ]:
df_test = df_test.join(df_store, on='Store')

In [ ]:
df_test.drop(['Id', 'Date', 'Year'], axis=1, inplace=True)

In [ ]:
df_test.fillna(0, inplace=True)

In [ ]:
df_test.head().T

In [ ]:
df_test.Open.unique()

In [ ]:
ind_open = df_test.Open == 1
ind_closed = df_test.Open == 0
df_test2 = df_test[ind_open]

In [ ]:
df_test2.drop('Open', axis=1, inplace=True)

In [ ]:
df_test2.head().T

In [ ]:
X_out = df_test2.values

In [ ]:
y_out = model1.predict(X_out)

In [ ]:
df_out = pd.DataFrame(np.zeros(len(df_test)), columns=['Sales'])

In [ ]:
df_out[ind_open] = y_out.reshape(-1,1)

In [ ]:
df_out.set_index(np.arange(len(df_out))+1, inplace=True)

In [ ]:
df_out.index.name = 'Id'

In [ ]:
df_out.head()

In [ ]:
df_out.to_csv('out5.csv')

Скор из submissions: 0.15383